In [ ]:
# this file created the table in postgres

In [1]:
import sqlalchemy as db
import pandas as pd
import numpy as np
import psycopg2 as p

In [2]:
df = pd.read_csv('resources/yelp_test1.csv')

In [3]:
df = df.rename(columns={'Diversity':'diversity'})

In [4]:
#postal_code is the wrong dtype - dtype must be int
df.dtypes

business_id      object
city             object
state            object
postal_code      object
stars           float64
review_count      int64
diversity         int64
dtype: object

In [5]:
#total number pre-change
df.postal_code.count()

34984

In [6]:
#length of the columns in postgres
print(df.business_id.map(len).max())
print(df.city.map(len).max())
print(df.state.map(len).max())

22
52
3


In [7]:
#number of nulls
df.isna().sum()

business_id     0
city            0
state           0
postal_code     4
stars           0
review_count    0
diversity       0
dtype: int64

In [8]:
# filter out the non-null postal_codes
df = df[pd.to_numeric(df['postal_code'], errors='coerce').notnull()]
df[df['postal_code'].isnull()].count()

business_id     0
city            0
state           0
postal_code     0
stars           0
review_count    0
diversity       0
dtype: int64

In [9]:
def check_int(value):
    try:
        int(value)
        return np.NaN
    except ValueError:
        return value

In [10]:
# non-int postal codes = Canada
df = df[df.postal_code.apply(check_int).isna()]

In [11]:
df['postal_code'] = df['postal_code'].astype(int)
df.dtypes

business_id      object
city             object
state            object
postal_code       int32
stars           float64
review_count      int64
diversity         int64
dtype: object

In [12]:
#replace all commas in a cell with nothing
df.replace(',','', regex=True, inplace=True)

In [13]:
df.to_csv('postgres_upload_file.csv', index=False)

In [16]:
# Connecting to PostgreSQL server at localhost using psycopg2 DBAPI 
connection = p.connect(
    host = "restaurant-and-house-prices.cx3jpmtuzudq.us-west-1.rds.amazonaws.com",
    port = 5432,
    user = "postgres",
    password = "",
    database = ""
)
cursor=connection.cursor()

In [17]:
cursor.execute(""" CREATE TABLE yelp_test1(
business_id text PRIMARY KEY,
city text,
state text,
postal_code integer,
stars float,
reveiw_count integer,
diversity integer)""")

connection.commit()

In [ ]:
with open('postgres_upload_file.csv', 'r') as row:
    next(row)# Skip the header row
    cursor.copy_from(row,'yelp_test1',sep=',')